## COD API, Matches --> clean & reshape

#### import stuff to work with

In [1]:
import asyncio
import os
import dotenv
from pprint import pprint
import datetime
from datetime import datetime, timezone, timedelta
import pandas as pd
import numpy as np
import pickle


import callofduty
from callofduty import Mode, Platform, Title, TimeFrame, GameType

In [2]:
# Using SSO
# We're storing our SSO token in an .env file stored locally to separate our config from code (w. python-dotenv). An.env-template file (with help to retrieve token) is provided for you to edit and populate the variable(s)
# callofduty.py client .Login() goes through all the authentification steps and initiate a session to access protected routes
# The client is asynchronous thus the 'await style'
from dotenv import load_dotenv
load_dotenv()
client = await callofduty.Login(sso=os.environ["SSO"])

In [3]:
# This time we're adding additional methods in the Call of Duty .py client only as there is no need to modify the HTTP class that already contains the endpoint we want to use
# Import the Class we want to modify

import urllib.parse
from typing import List, Optional, Union

from callofduty.client import Client
from callofduty.http import HTTP
from callofduty.http import Request


# following additional methods to be added in callofduty.client.py Client Class
# see notebooks/cod_api_doc.ipnyb for details

async def GetMatches(
    self, platform, username: str, title: Title, mode: Mode, **kwargs
):

    limit: int = kwargs.get("limit", 20)
    startTimestamp: int = kwargs.get("startTimestamp", 0)
    endTimestamp: int = kwargs.get("endTimestamp", 0)

    data: dict = (
        await self.http.GetPlayerMatches(
            platform,
            username,
            title.value,
            mode.value,
            limit,
            startTimestamp,
            endTimestamp,
        )
    )["data"] # API res was filtered out here

    return data


async def GetMatchesDetailed(
    self, platform, username: str, title: Title, mode: Mode, **kwargs
):

    limit: int = kwargs.get("limit", 20)
    startTimestamp: int = kwargs.get("startTimestamp", 0)
    endTimestamp: int = kwargs.get("endTimestamp", 0)

    return (
        await self.http.GetPlayerMatchesDetailed(
            platform,
            username,
            title.value,
            mode.value,
            limit,
            startTimestamp,
            endTimestamp,
        )
    )["data"]['matches'] # API res was filtered out here


async def GetMatchesSummary(
    self, platform, username: str, title: Title, mode: Mode, **kwargs
):

    limit: int = kwargs.get("limit", 20)
    startTimestamp: int = kwargs.get("startTimestamp", 0)
    endTimestamp: int = kwargs.get("endTimestamp", 0)

    return (
        await self.http.GetPlayerMatchesDetailed(
            platform,
            username,
            title.value,
            mode.value,
            limit,
            startTimestamp,
            endTimestamp,
        )
    )["data"]['summary'] # API res was filtered out here


# add our modified methods into callofduty Client Class

Client.GetMatches = GetMatches
Client.GetMatchesDetailed = GetMatchesDetailed
Client.GetMatchesSummary = GetMatchesSummary

# raw_matches_datailed has indeed two keys : summary and matches (the one that is filtered out in the Client)
matches = await client.GetMatchesDetailed("battle", "AMADEVS#1689", Title.ModernWarfare, Mode.Warzone, limit=20)


In [4]:
# save file if you want to work in offline mode
#with open("matches.pkl", 'wb') as f:
#    pickle.dump(matches, f)

### Overview of returned stats

In [2]:
# load offline data
open_file = open("matches.pkl", "rb")
loaded_matches = pickle.load(open_file)
open_file.close()

#### raw data

In [3]:
raw = pd.DataFrame(loaded_matches)
raw.head(5)

,utcStartSeconds,utcEndSeconds,map,mode,matchID,duration,playlistName,version,gameType,playerCount,playerStats,player,teamCount,rankedTeams,draw,privateMatch
0,1634595215,1634596737,mp_don4,br_brduos,9207377216339591388,1522000,None,1,wz,150,"{'kills': 0.0, 'medalXp': 10.0, 'matchXp': 875...","{'team': 'team_forty', 'rank': 54.0, 'awards':...",77,None,False,False
1,1634594468,1634596060,mp_don4,br_brduos,7178723139144849769,1592000,None,1,wz,150,"{'kills': 1.0, 'medalXp': 10.0, 'matchXp': 539...","{'team': 'team_ten', 'rank': 54.0, 'awards': {...",76,None,False,False
2,1634593656,1634595328,mp_don4,br_brduos,13958697112921893137,1672000,None,1,wz,151,"{'kills': 3.0, 'medalXp': 40.0, 'objectiveTeam...","{'team': 'team_twenty', 'rank': 54.0, 'awards'...",77,None,False,False
3,1634593229,1634594881,mp_don4,br_brduos,8474707468644121647,1652000,None,1,wz,149,"{'kills': 0.0, 'medalXp': 20.0, 'matchXp': 200...","{'team': 'team_forty_nine', 'rank': 54.0, 'awa...",76,None,False,False
4,1634592494,1634594041,mp_don4,br_brduos,8088877590056611003,1547000,None,1,wz,151,"{'kills': 1.0, 'medalXp': 20.0, 'matchXp': 570...","{'team': 'team_twenty_six', 'rank': 54.0, 'awa...",75,None,False,False


In [4]:
raw.keys()

Index(['utcStartSeconds', 'utcEndSeconds', 'map', 'mode', 'matchID',
       'duration', 'playlistName', 'version', 'gameType', 'playerCount',
       'playerStats', 'player', 'teamCount', 'rankedTeams', 'draw',
       'privateMatch'],
      dtype='object')

#### playerStats col

In [5]:
player_stats = raw['playerStats'].apply(pd.Series)
player_stats.head(20)

,kills,medalXp,matchXp,scoreXp,wallBangs,score,totalXp,headshots,assists,challengeXp,...,damageTaken,objectiveTeamWiped,objectiveLastStandKill,objectiveBrDownEnemyCircle1,objectiveReviver,objectiveDestroyedEquipment,objectiveMunitionsBoxTeammateUsed,objectiveBrDownEnemyCircle5,objectiveBrDownEnemyCircle4,objectiveBrDownEnemyCircle6
0,0.0,10.0,8754.0,2950.0,0.0,2100.0,11714.0,0.0,0.0,0.0,...,436.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1.0,10.0,5391.0,2700.0,0.0,1850.0,8101.0,0.0,0.0,0.0,...,356.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3.0,40.0,6056.0,3800.0,0.0,1900.0,9896.0,2.0,0.0,0.0,...,402.0,1.0,2.0,2.0,1.0,NaN,NaN,NaN,NaN,NaN
3,0.0,20.0,2008.0,1025.0,0.0,325.0,3053.0,0.0,1.0,0.0,...,350.0,NaN,NaN,1.0,1.0,NaN,NaN,NaN,NaN,NaN
4,1.0,20.0,5707.0,1550.0,0.0,775.0,7277.0,0.0,4.0,0.0,...,429.0,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
5,2.0,110.0,8769.0,1825.0,0.0,1200.0,10762.0,1.0,2.0,0.0,...,738.0,1.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN
6,1.0,20.0,6797.0,1475.0,0.0,875.0,8292.0,0.0,0.0,0.0,...,534.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN
7,1.0,40.0,2061.0,850.0,0.0,650.0,2951.0,0.0,0.0,0.0,...,322.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,8.0,230.0,4107.0,1100.0,0.0,1100.0,5437.0,7.0,0.0,0.0,...,565.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,11.0,480.0,7400.0,1650.0,0.0,1650.0,9530.0,5.0,6.0,0.0,...,1230.0,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN


In [6]:
player_stats.keys()

Index(['kills', 'medalXp', 'matchXp', 'scoreXp', 'wallBangs', 'score',
       'totalXp', 'headshots', 'assists', 'challengeXp', 'rank',
       'scorePerMinute', 'distanceTraveled', 'teamSurvivalTime', 'deaths',
       'kdRatio', 'objectiveBrMissionPickupTablet', 'bonusXp',
       'objectiveBrKioskBuy', 'gulagDeaths', 'timePlayed', 'executions',
       'gulagKills', 'nearmisses', 'objectiveBrCacheOpen', 'percentTimeMoving',
       'miscXp', 'longestStreak', 'teamPlacement', 'damageDone', 'damageTaken',
       'objectiveTeamWiped', 'objectiveLastStandKill',
       'objectiveBrDownEnemyCircle1', 'objectiveReviver',
       'objectiveDestroyedEquipment', 'objectiveMunitionsBoxTeammateUsed',
       'objectiveBrDownEnemyCircle5', 'objectiveBrDownEnemyCircle4',
       'objectiveBrDownEnemyCircle6'],
      dtype='object')

In [7]:
player_stats[['headshots', 'distanceTraveled', 'teamSurvivalTime', 'objectiveBrKioskBuy']].head(5)

,headshots,distanceTraveled,teamSurvivalTime,objectiveBrKioskBuy
0,0.0,200747.84,793296.0,1.0
1,0.0,341707.75,595776.0,1.0
2,2.0,254221.58,544752.0,NaN
3,0.0,205961.95,269664.0,NaN
4,0.0,272237.56,578112.0,NaN


#### player col

In [8]:
player = raw['player'].apply(pd.Series)
player.head(5)

,team,rank,awards,username,uno,clantag,brMissionStats,loadout
0,team_forty,54.0,{},gentil_renard,2621859779580650696,lkf :,"{'missionsComplete': 0, 'totalMissionXpEarned'...","[{'primaryWeapon': {'name': 'iw8_sh_oscar12', ..."
1,team_ten,54.0,{},gentil_renard,2621859779580650696,lkf :,"{'missionsComplete': 1, 'totalMissionXpEarned'...","[{'primaryWeapon': {'name': 'iw8_sh_oscar12', ..."
2,team_twenty,54.0,{},gentil_renard,2621859779580650696,lkf :,"{'missionsComplete': 1, 'totalMissionXpEarned'...","[{'primaryWeapon': {'name': 'iw8_sh_oscar12', ..."
3,team_forty_nine,54.0,{},gentil_renard,2621859779580650696,lkf :,"{'missionsComplete': 0, 'totalMissionXpEarned'...","[{'primaryWeapon': {'name': 'iw8_sh_oscar12', ..."
4,team_twenty_six,54.0,{},gentil_renard,2621859779580650696,lkf :,"{'missionsComplete': 0, 'totalMissionXpEarned'...","[{'primaryWeapon': {'name': 'iw8_sh_oscar12', ..."


In [9]:
player.keys()

Index(['team', 'rank', 'awards', 'username', 'uno', 'clantag',
       'brMissionStats', 'loadout'],
      dtype='object')

#### Deep dive into player.loadout entry

In [10]:
# Each entry of 'loadout' is a list of dict. Either one dict (if 1 loadout) or more(2 or more loadouts taken)
# Example, matche at index 11 with two loadouts uses (one list with 2 dict).
player['loadout'][11]

[{'primaryWeapon': {'name': 'iw8_sh_oscar12',
   'label': None,
   'imageLoot': None,
   'imageIcon': None,
   'variant': '0',
   'attachments': [{'name': 'barmid',
     'label': None,
     'image': None,
     'category': None},
    {'name': 'xmags', 'label': None, 'image': None, 'category': None},
    {'name': 'laserrange', 'label': None, 'image': None, 'category': None},
    {'name': 'stockno', 'label': None, 'image': None, 'category': None},
    {'name': 'silencer3', 'label': None, 'image': None, 'category': None}]},
  'secondaryWeapon': {'name': 'iw8_sn_t9powersemi',
   'label': None,
   'imageLoot': None,
   'imageIcon': None,
   'variant': '13',
   'attachments': [{'name': 'silencer',
     'label': None,
     'image': None,
     'category': None},
    {'name': 'laserrange', 'label': None, 'image': None, 'category': None},
    {'name': 'acog3', 'label': None, 'image': None, 'category': None},
    {'name': 'pistolgrip06', 'label': None, 'image': None, 'category': None},
    {'name'

In [11]:
# first loadout structure 
player['loadout'][11][1].keys()

dict_keys(['primaryWeapon', 'secondaryWeapon', 'perks', 'extraPerks', 'killstreaks', 'tactical', 'lethal'])

In [12]:
player['loadout'][11][1]['primaryWeapon'].keys()

dict_keys(['name', 'label', 'imageLoot', 'imageIcon', 'variant', 'attachments'])

#### expanded data : raw + playerStats & player columns expanded + player.loadout expanded

In [13]:
expanded = pd.DataFrame(loaded_matches)
expanded = pd.concat([expanded.drop(['playerStats'], axis=1), expanded['playerStats'].apply(pd.Series)], axis=1)
expanded = pd.concat([expanded.drop(['player'], axis=1), expanded['player'].apply(pd.Series)], axis=1)
expanded = expanded.drop(['brMissionStats'], axis = 1)

# player.loadout column
expanded = pd.concat([expanded.drop(['loadout'], axis=1), expanded['loadout'].apply(pd.Series)], axis=1)
for col in range(0,3):
    if col in expanded.columns:
        expanded[col] = expanded[col].apply(lambda x: [x['primaryWeapon']['name'],x['secondaryWeapon']['name']] if not str(x) == 'nan' else np.nan)
        expanded = expanded.rename(columns={col: f"loadout_{str(col +1)}"})
expanded.head(15)

,utcStartSeconds,utcEndSeconds,map,mode,matchID,duration,playlistName,version,gameType,playerCount,...,rank,awards,username,uno,clantag,loadout_1,loadout_2,loadout_3,3,4
0,1634595215,1634596737,mp_don4,br_brduos,9207377216339591388,1522000,None,1,wz,150,...,54.0,{},gentil_renard,2621859779580650696,lkf :,"[iw8_sh_oscar12, iw8_sn_t9accurate]","[iw8_ar_t9longburst, iw8_fists]",NaN,NaN,NaN
1,1634594468,1634596060,mp_don4,br_brduos,7178723139144849769,1592000,None,1,wz,150,...,54.0,{},gentil_renard,2621859779580650696,lkf :,"[iw8_sh_oscar12, iw8_sn_t9accurate]",NaN,NaN,NaN,NaN
2,1634593656,1634595328,mp_don4,br_brduos,13958697112921893137,1672000,None,1,wz,151,...,54.0,{},gentil_renard,2621859779580650696,lkf :,"[iw8_sh_oscar12, iw8_sn_t9accurate]",NaN,NaN,NaN,NaN
3,1634593229,1634594881,mp_don4,br_brduos,8474707468644121647,1652000,None,1,wz,149,...,54.0,{},gentil_renard,2621859779580650696,lkf :,"[iw8_sh_oscar12, iw8_sn_t9accurate]","[iw8_ar_t9fastburst, iw8_fists]",NaN,NaN,NaN
4,1634592494,1634594041,mp_don4,br_brduos,8088877590056611003,1547000,None,1,wz,151,...,54.0,{},gentil_renard,2621859779580650696,lkf :,"[iw8_sh_oscar12, iw8_sn_t9accurate]",NaN,NaN,NaN,NaN
5,1634591434,1634593044,mp_don4,br_brduos,17736843437097054620,1610000,None,1,wz,152,...,54.0,{},gentil_renard,2621859779580650696,lkf :,"[iw8_sh_oscar12, iw8_sn_t9accurate]",NaN,NaN,NaN,NaN
6,1634590494,1634592109,mp_don4,br_brduos,16948376130339996479,1615000,None,1,wz,151,...,54.0,{},gentil_renard,2621859779580650696,lkf :,"[iw8_sh_oscar12, iw8_sn_t9accurate]","[iw8_sh_t9pump, iw8_fists]",NaN,NaN,NaN
7,1634590115,1634591656,mp_don4,br_brduos,15022667824293845519,1541000,None,1,wz,147,...,54.0,{},gentil_renard,2621859779580650696,lkf :,"[iw8_sh_oscar12, iw8_sn_t9accurate]",NaN,NaN,NaN,NaN
8,1634589285,1634590190,mp_don4,br_rumble_clash,2468067755055017928,905000,None,1,wz,101,...,54.0,{},gentil_renard,2621859779580650696,lkf :,"[iw8_sh_oscar12, iw8_sn_t9accurate]",NaN,NaN,NaN,NaN
9,1634588223,1634589112,mp_don4,br_rumble_clash,1229923170792459222,889000,None,1,wz,100,...,54.0,{},gentil_renard,2621859779580650696,lkf :,"[iw8_sh_oscar12, iw8_sn_t9powersemi]","[iw8_sh_oscar12, iw8_sn_t9accurate]",NaN,NaN,NaN


In [14]:
def MatchesToDf(matches):
    """
    COD API / callofduty.py client --> list of dict : max 20 Matches with Player stats
    We expand Player and PlayerStats levels, concatenate them
    ! We drop some cols we dont want to work with

    Returns
    -------
    DataFrame, matches as rows, matches/player stats as columns
    """
    
    
    keep_cols =  [
        'mode',
        'utcStartSeconds',
        'utcEndSeconds',
        'timePlayed',
        'teamPlacement',
        'kdRatio', 
        'kills', 
        'deaths', 
        'assists', 
        'damageDone',
        'damageTaken',
        'gulagKills',
        'percentTimeMoving',
        'duration'
        ]

    df = pd.DataFrame(matches)
    
    # column playerStats is a series of dict, we can expand it easily and append, then drop the original
    df = pd.concat([df.drop(['playerStats'], axis=1), df['playerStats'].apply(pd.Series)], axis=1)
    
    # colum player has more depth
    # once expanded, it has a column 'loadout' : a series of list of dict (either one or more, we will keep 1 to max 3)
    # and also brMissionStats (mostly empty ?) that we aren't interested in
    
    df = pd.concat([df.drop(['player'], axis=1), df['player'].apply(pd.Series)], axis=1)
    df = df.drop(['brMissionStats'], axis = 1)
    df = pd.concat([df.drop(['loadout'], axis=1), df['loadout'].apply(pd.Series)], axis=1)
    for col in range(0,3):
        if col in df.columns:
            df[col] = df[col].apply(lambda x: f"{x['primaryWeapon']['name']} - {x['secondaryWeapon']['name']}" if not str(x) == 'nan' else np.nan)
            col_name = "loadout_" + str(col +1)
            df = df.rename(columns={col: f"loadout_{str(col +1)}"})   
            keep_cols.append(col_name)
    
    return df[keep_cols]

In [15]:
raw = pd.DataFrame(loaded_matches)
df = MatchesToDf(raw)
df.head(3)

,mode,utcStartSeconds,utcEndSeconds,timePlayed,teamPlacement,kdRatio,kills,deaths,assists,damageDone,damageTaken,gulagKills,percentTimeMoving,duration,loadout_1,loadout_2,loadout_3
0,br_brduos,1634595215,1634596737,877.0,18.0,0.0,0.0,3.0,0.0,281.0,436.0,0.0,83.021225,1522000,iw8_sh_oscar12 - iw8_sn_t9accurate,iw8_ar_t9longburst - iw8_fists,NaN
1,br_brduos,1634594468,1634596060,674.0,44.0,0.5,1.0,2.0,0.0,418.0,356.0,1.0,98.484840,1592000,iw8_sh_oscar12 - iw8_sn_t9accurate,NaN,NaN
2,br_brduos,1634593656,1634595328,650.0,35.0,1.5,3.0,2.0,0.0,774.0,402.0,1.0,70.524414,1672000,iw8_sh_oscar12 - iw8_sn_t9accurate,NaN,NaN


In [16]:
df

,mode,utcStartSeconds,utcEndSeconds,timePlayed,teamPlacement,kdRatio,kills,deaths,assists,damageDone,damageTaken,gulagKills,percentTimeMoving,duration,loadout_1,loadout_2,loadout_3
0,br_brduos,1634595215,1634596737,877.0,18.0,0.000000,0.0,3.0,0.0,281.0,436.0,0.0,83.021225,1522000,iw8_sh_oscar12 - iw8_sn_t9accurate,iw8_ar_t9longburst - iw8_fists,NaN
1,br_brduos,1634594468,1634596060,674.0,44.0,0.500000,1.0,2.0,0.0,418.0,356.0,1.0,98.484840,1592000,iw8_sh_oscar12 - iw8_sn_t9accurate,NaN,NaN
2,br_brduos,1634593656,1634595328,650.0,35.0,1.500000,3.0,2.0,0.0,774.0,402.0,1.0,70.524414,1672000,iw8_sh_oscar12 - iw8_sn_t9accurate,NaN,NaN
3,br_brduos,1634593229,1634594881,351.0,65.0,0.000000,0.0,3.0,1.0,264.0,350.0,0.0,70.143880,1652000,iw8_sh_oscar12 - iw8_sn_t9accurate,iw8_ar_t9fastburst - iw8_fists,NaN
4,br_brduos,1634592494,1634594041,659.0,40.0,1.000000,1.0,1.0,4.0,791.0,429.0,0.0,90.753426,1547000,iw8_sh_oscar12 - iw8_sn_t9accurate,NaN,NaN
5,br_brduos,1634591434,1634593044,961.0,18.0,1.000000,2.0,2.0,2.0,841.0,738.0,1.0,92.613640,1610000,iw8_sh_oscar12 - iw8_sn_t9accurate,NaN,NaN
6,br_brduos,1634590494,1634592109,716.0,31.0,0.333333,1.0,3.0,0.0,659.0,534.0,0.0,81.571000,1615000,iw8_sh_oscar12 - iw8_sn_t9accurate,iw8_sh_t9pump - iw8_fists,NaN
7,br_brduos,1634590115,1634591656,341.0,61.0,0.500000,1.0,2.0,0.0,494.0,322.0,1.0,94.230774,1541000,iw8_sh_oscar12 - iw8_sn_t9accurate,NaN,NaN
8,br_rumble_clash,1634589285,1634590190,568.0,1.0,1.600000,8.0,5.0,0.0,1746.0,565.0,0.0,94.845360,905000,iw8_sh_oscar12 - iw8_sn_t9accurate,NaN,NaN
9,br_rumble_clash,1634588223,1634589112,988.0,2.0,1.000000,11.0,11.0,6.0,3202.0,1230.0,0.0,90.022940,889000,iw8_sh_oscar12 - iw8_sn_t9powersemi,iw8_sh_oscar12 - iw8_sn_t9accurate,NaN


In [17]:
def MatchesStandardize(df):
    """
    A first layer of standadization (as properly formated) to our matches DataFrame
    For further aggregations / better readibility of our data
    
    Returns
    -------
    DataFrame : matches as rows, cleaned matches/player stats as columns
    """
    
    int_cols =  [
        'teamPlacement', 
        'kills', 
        'deaths', 
        'assists', 
        'gulagKills', 
        'damageDone',
        'damageTaken'
        ]
    
    float_cols = [
        'kdRatio',
        'percentTimeMoving'
        ]
    
    ts_cols = [
        'utcStartSeconds',
        'utcEndSeconds'
        ]
    
    mode_labels = {
        'br_brtrios':'Trios',
        'br_brduos':'Duos',
        'br_brquads':'Quads',
        'br_dbd_dbd':'Iron Trials',
        'br_dmz_plunquad':'Pldr x4',
        'br_rumble_clash':'Rumble'

        }
    
    columns_labels = {
        'utcEndSeconds':'Ended at',
        'utcStartSeconds':'Started at',
        'timePlayed': 'Playtime',
        'teamPlacement':'#',
        'kdRatio':'KD',
        'damageDone':'Damage >',
        'damageTaken':'Damage <',
        'gulagKills':'Gulag',
        'headshots':'% headshots',
        'percentTimeMoving':'% moving',
        'duration':'Game duration'
        }

    # df = df.fillna(0)
    df[int_cols] = df[int_cols].astype(int)
    df[float_cols] = df[float_cols].astype(float).round(1) # still renders 0.0000 in streamlit but ugly hacks exists
    
    # specials
    df['utcEndSeconds'] = df['utcEndSeconds'].apply(lambda x: datetime.fromtimestamp(x))
    df['utcStartSeconds'] = df['utcStartSeconds'].apply(lambda x: datetime.fromtimestamp(x))
    
    df['duration'] = df['duration'].apply(lambda x: x/1000).apply(lambda x: pd.to_datetime(x, unit='s').strftime('%M')) # API duration is in seconds x1000
    df['timePlayed'] = df['timePlayed'].apply(lambda x: pd.to_datetime(x, unit='s').strftime('%M:%S')) # API timePlayed is in seconds
    df['gulagKills'] = df['gulagKills'].map({1:'W', 0:'L'})
    for col in ['loadout_1', 'loadout_2', 'loadout_3']:
            df.fillna({col:'-'}, inplace=True) if col in df.columns else None
    
    df = df.replace({"mode": mode_labels})
    df = df.rename(columns=columns_labels)
    df.columns = df.columns.str.capitalize()
    df = df.rename({"Kd":"KD"}, axis=1)
     
    return df

In [18]:
raw = pd.DataFrame(loaded_matches)
df = MatchesToDf(raw)
df = MatchesStandardize(df)
df.head(5)

,Mode,Started at,Ended at,Playtime,#,KD,Kills,Deaths,Assists,Damage >,Damage <,Gulag,% moving,Game duration,Loadout_1,Loadout_2,Loadout_3
0,Duos,2021-10-19 00:13:35,2021-10-19 00:38:57,14:37,18,0.0,0,3,0,281,436,L,83.0,25,iw8_sh_oscar12 - iw8_sn_t9accurate,iw8_ar_t9longburst - iw8_fists,-
1,Duos,2021-10-19 00:01:08,2021-10-19 00:27:40,11:14,44,0.5,1,2,0,418,356,W,98.5,26,iw8_sh_oscar12 - iw8_sn_t9accurate,-,-
2,Duos,2021-10-18 23:47:36,2021-10-19 00:15:28,10:50,35,1.5,3,2,0,774,402,W,70.5,27,iw8_sh_oscar12 - iw8_sn_t9accurate,-,-
3,Duos,2021-10-18 23:40:29,2021-10-19 00:08:01,05:51,65,0.0,0,3,1,264,350,L,70.1,27,iw8_sh_oscar12 - iw8_sn_t9accurate,iw8_ar_t9fastburst - iw8_fists,-
4,Duos,2021-10-18 23:28:14,2021-10-18 23:54:01,10:59,40,1.0,1,1,4,791,429,L,90.8,25,iw8_sh_oscar12 - iw8_sn_t9accurate,-,-


In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 17 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   Mode           20 non-null     object        
 1   Started at     20 non-null     datetime64[ns]
 2   Ended at       20 non-null     datetime64[ns]
 3   Playtime       20 non-null     object        
 4   #              20 non-null     int64         
 5   KD             20 non-null     float64       
 6   Kills          20 non-null     int64         
 7   Deaths         20 non-null     int64         
 8   Assists        20 non-null     int64         
 9   Damage >       20 non-null     int64         
 10  Damage <       20 non-null     int64         
 11  Gulag          20 non-null     object        
 12  % moving       20 non-null     float64       
 13  Game duration  20 non-null     object        
 14  Loadout_1      20 non-null     object        
 15  Loadout_2      20 non-nul

In [20]:
loadout_cols = df.columns[df.columns.str.startswith('loadout')].tolist()

In [21]:
def MatchesPerDay(df):
    """
    Final layer applied to our list of matches with stats
    Streamlit/basic AgGrid does not render well (aka w. blank rows etc.) multi indexed df
    So we structure our data differently, in a daily manner
    
    Returns
    -------
    Dictionary :
    {
        "str_weekday_1":df-of-matches-that-day,
        "str_weekday_2":df-of-matches-that-day,
        (...)
    }
    """
    
    drop_cols =  [
        'Started at',
        'Playtime',
        '% moving',
        'Game duration'
        ]
    
    keep_cols = [
        'End time',
        'Mode',
        '#',
        'KD',
        'Kills',
        'Deaths',
        'Assists',
        'Damage >',
        'Damage <',
        'Gulag'
    ]
    
    loadout_cols = df.columns[df.columns.str.startswith('Loadout')].tolist()
    
    # 1. --- initial formating : datetime, concat loadouts cols ---
    
    df = df.drop(drop_cols, axis = 1)
    df['End time'] = df['Ended at'].dt.time
    
    def concat_loadouts(df, columns):
        return pd.Series(map(' , '.join, df[columns].values.tolist()),index = df.index)
    
    df['Weapons'] = concat_loadouts(df, loadout_cols)
    keep_cols = [*keep_cols, *['Weapons']]
    
    # 2. --- build result {"day1": df-matches-that-day, "day2": df...} ---
    
    list_df = [g for n, g in df.groupby(pd.Grouper(key='Ended at',freq='D'))]
    list_df = [df for df in list_df if not df.empty]
    list_days = [df['Ended at'].tolist()[0].strftime('%A') for df in list_df]
    
    # make sure we display latest day first, then build dictionary
    for list_ in [list_days, list_df]:
        list_.reverse()
    day_matches = dict(zip(list_days, list_df))
    
    # 3. --- some more (re)formating ---
    
    # for some reason (me ? ^-^, Grouper => Series?) couldnt' modify df before building the result, must iterate again 
    for k, v in day_matches.items():
        day_matches[k] = day_matches[k][keep_cols]
      
    return day_matches


def AggStats(df):
    """
    We want to add aggregated stats for each day/df of matches we got from MatchesPerDay()
    Each daily aggregation will be rendered in our Front on top of each list of matches
    
    Returns
    -------
    Dictionary :
    {
    'Kills':total n kills that day,
    'Deaths': total deaths,
    'KD': kills/deaths,
    'Gulags': win % ,
    'Played': count matches -
    }
    """

    agg_func = {
        "Mode":"count",
        "Kills":"sum",
        "Deaths":"sum"
    } 
    dict_ = df.agg(agg_func).to_dict()
    dict_.update({'KD': (df.Kills.sum() / df.Deaths.sum()).round(2)})
    dict_.update({'Gulags': int((df.Gulag.str.count("W").sum() / df.Gulag.str.count("L").sum())*100)})
    dict_['Played'] = dict_.pop('Mode')
    
    
    return dict_

In [22]:
daily = MatchesPerDay(df)
for day in daily.keys():
    print(day)
    display(daily[day])
    print(daily[day]['Mode'].values)

Tuesday


,End time,Mode,#,KD,Kills,Deaths,Assists,Damage >,Damage <,Gulag,Weapons
3,00:08:01,Duos,65,0.0,0,3,1,264,350,L,"iw8_sh_oscar12 - iw8_sn_t9accurate , iw8_ar_t9..."
2,00:15:28,Duos,35,1.5,3,2,0,774,402,W,"iw8_sh_oscar12 - iw8_sn_t9accurate , - , -"
1,00:27:40,Duos,44,0.5,1,2,0,418,356,W,"iw8_sh_oscar12 - iw8_sn_t9accurate , - , -"
0,00:38:57,Duos,18,0.0,0,3,0,281,436,L,"iw8_sh_oscar12 - iw8_sn_t9accurate , iw8_ar_t9..."


['Duos' 'Duos' 'Duos' 'Duos']
Monday


,End time,Mode,#,KD,Kills,Deaths,Assists,Damage >,Damage <,Gulag,Weapons
14,19:27:11,Rumble,1,1.1,8,7,5,2436,773,L,"iw8_sh_oscar12 - iw8_lm_mgolf34 , iw8_sh_oscar..."
13,19:55:13,Rumble,1,0.4,4,9,4,1372,1042,L,"iw8_sh_oscar12 - iw8_lm_mgolf34 , iw8_sh_oscar..."
12,20:15:46,Rumble,2,0.5,4,8,1,1460,987,L,"iw8_sh_oscar12 - iw8_sn_t9powersemi , iw8_sh_o..."
11,20:41:40,Duos,55,0.5,1,2,0,114,267,W,"iw8_sh_oscar12 - iw8_sn_t9powersemi , iw8_pi_t..."
10,20:50:10,Duos,5,1.3,4,3,1,1239,744,L,"iw8_sh_oscar12 - iw8_sn_t9powersemi , iw8_pi_t..."
9,22:31:52,Rumble,2,1.0,11,11,6,3202,1230,L,"iw8_sh_oscar12 - iw8_sn_t9powersemi , iw8_sh_o..."
8,22:49:50,Rumble,1,1.6,8,5,0,1746,565,L,"iw8_sh_oscar12 - iw8_sn_t9accurate , - , -"
7,23:14:16,Duos,61,0.5,1,2,0,494,322,W,"iw8_sh_oscar12 - iw8_sn_t9accurate , - , -"
6,23:21:49,Duos,31,0.3,1,3,0,659,534,L,"iw8_sh_oscar12 - iw8_sn_t9accurate , iw8_sh_t9..."
5,23:37:24,Duos,18,1.0,2,2,2,841,738,W,"iw8_sh_oscar12 - iw8_sn_t9accurate , - , -"


['Rumble' 'Rumble' 'Rumble' 'Duos' 'Duos' 'Rumble' 'Rumble' 'Duos' 'Duos'
 'Duos' 'Duos']
Saturday


,End time,Mode,#,KD,Kills,Deaths,Assists,Damage >,Damage <,Gulag,Weapons
19,00:48:01,Iron Trials,1,2.5,5,2,3,1939,1540,L,"iw8_sh_oscar12 - iw8_sn_t9accurate , iw8_ar_t9..."
18,01:13:57,Iron Trials,1,1.5,3,2,1,2168,1568,L,"iw8_sh_oscar12 - iw8_sn_t9accurate , iw8_ar_t9..."
17,01:41:31,Iron Trials,41,0.5,1,2,0,286,678,W,"iw8_sh_oscar12 - iw8_sn_t9accurate , - , -"
16,20:14:11,Rumble,1,0.6,3,5,7,1543,504,L,"iw8_sh_oscar12 - iw8_lm_kilo121 , iw8_sh_oscar..."
15,20:31:46,Rumble,2,1.7,17,10,4,4493,1105,L,"iw8_sh_oscar12 - iw8_sn_t9accurate , - , -"


['Iron Trials' 'Iron Trials' 'Iron Trials' 'Rumble' 'Rumble']


In [23]:
for k, v in daily.items():
    print(k)
    print(AggStats(v))

Tuesday
{'Kills': 4, 'Deaths': 10, 'KD': 0.4, 'Gulags': 100, 'Played': 4}
Monday
{'Kills': 45, 'Deaths': 53, 'KD': 0.85, 'Gulags': 37, 'Played': 11}
Saturday
{'Kills': 29, 'Deaths': 21, 'KD': 1.38, 'Gulags': 25, 'Played': 5}


In [32]:
df = daily["Monday"].copy(deep=True)
df

,End time,Mode,#,KD,Kills,Deaths,Assists,Damage >,Damage <,Gulag,Weapons
14,19:27:11,Rumble,1,1.1,8,7,5,2436,773,L,"iw8_sh_oscar12 - iw8_lm_mgolf34 , iw8_sh_oscar..."
13,19:55:13,Rumble,1,0.4,4,9,4,1372,1042,L,"iw8_sh_oscar12 - iw8_lm_mgolf34 , iw8_sh_oscar..."
12,20:15:46,Rumble,2,0.5,4,8,1,1460,987,L,"iw8_sh_oscar12 - iw8_sn_t9powersemi , iw8_sh_o..."
11,20:41:40,Duos,55,0.5,1,2,0,114,267,W,"iw8_sh_oscar12 - iw8_sn_t9powersemi , iw8_pi_t..."
10,20:50:10,Duos,5,1.3,4,3,1,1239,744,L,"iw8_sh_oscar12 - iw8_sn_t9powersemi , iw8_pi_t..."
9,22:31:52,Rumble,2,1.0,11,11,6,3202,1230,L,"iw8_sh_oscar12 - iw8_sn_t9powersemi , iw8_sh_o..."
8,22:49:50,Rumble,1,1.6,8,5,0,1746,565,L,"iw8_sh_oscar12 - iw8_sn_t9accurate , - , -"
7,23:14:16,Duos,61,0.5,1,2,0,494,322,W,"iw8_sh_oscar12 - iw8_sn_t9accurate , - , -"
6,23:21:49,Duos,31,0.3,1,3,0,659,534,L,"iw8_sh_oscar12 - iw8_sn_t9accurate , iw8_sh_t9..."
5,23:37:24,Duos,18,1.0,2,2,2,841,738,W,"iw8_sh_oscar12 - iw8_sn_t9accurate , - , -"


In [25]:
list_br_modes = ['Duos', 'Trios', 'Quads', 'Iron Trials', 'xxxx']
df[df['Mode'].isin(list_br_modes)]

,End time,Mode,#,KD,Kills,Deaths,Assists,Damage >,Damage <,Gulag,Weapons
19,00:48:01,Iron Trials,1,2.5,5,2,3,1939,1540,L,"iw8_sh_oscar12 - iw8_sn_t9accurate , iw8_ar_t9..."
18,01:13:57,Iron Trials,1,1.5,3,2,1,2168,1568,L,"iw8_sh_oscar12 - iw8_sn_t9accurate , iw8_ar_t9..."
17,01:41:31,Iron Trials,41,0.5,1,2,0,286,678,W,"iw8_sh_oscar12 - iw8_sn_t9accurate , - , -"


In [34]:
df['Mode'].unique().tolist()

['Rumble', 'Duos']